### Remove records with NULL customer_id

In [0]:
%sql
select customer_id,* from  democatalog.bronze.v_customers

In [0]:
%sql
select customer_id,* from  democatalog.bronze.v_customers
where customer_id is not null

### Remove Exact duplicates

In [0]:
%sql
select customer_id,* from  democatalog.bronze.v_customers
where customer_id is not null
order by customer_id

In [0]:
%sql
select distinct * from  democatalog.bronze.v_customers
where customer_id is not null
order by customer_id

In [0]:
%sql
create or replace temporary view v_customers_distinct
as
select distinct * from  democatalog.bronze.v_customers
where customer_id is not null
order by customer_id

In [0]:
%sql
select customer_id,
  max(created_timestamp) as max_created_timestamp
 from v_customers_distinct
 group by customer_id

In [0]:
%sql
with cte_max as 
(
    select customer_id,
         max(created_timestamp) as max_created_timestamp
    from v_customers_distinct
    group by customer_id
)
select t.*
    from v_customers_distinct t
    join cte_max m 
    on t.customer_id = m.customer_id 
    and t.created_timestamp = m.max_created_timestamp


### CAST the column values into correct data type

In [0]:
%sql
with cte_max as 
(
    select customer_id,
         max(created_timestamp) as max_created_timestamp
    from v_customers_distinct
    group by customer_id
)
select cast(t.created_timestamp as timestamp),
       t.customer_id,
       t.customer_name,
       cast(t.date_of_birth as date) as date_of_birth,
       t.email,
       cast(t.member_since as date) as member_since,
       t.telephone
    from v_customers_distinct t
    join cte_max m 
    on t.customer_id = m.customer_id 
    and t.created_timestamp = m.max_created_timestamp


### Write Data into a Delta Table

In [0]:
%sql
create table democatalog.silver.customers
as
with cte_max as 
(
    select customer_id,
         max(created_timestamp) as max_created_timestamp
    from v_customers_distinct
    group by customer_id
)
select cast(t.created_timestamp as timestamp),
       t.customer_id,
       t.customer_name,
       cast(t.date_of_birth as date) as date_of_birth,
       t.email,
       cast(t.member_since as date) as member_since,
       t.telephone
    from v_customers_distinct t
    join cte_max m 
    on t.customer_id = m.customer_id 
    and t.created_timestamp = m.max_created_timestamp


In [0]:
%sql
select * from democatalog.silver.customers